In [14]:
import requests
import time

In [15]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [16]:
data = requests.get(standings_url)
html_content = data.text

In [17]:
from bs4 import BeautifulSoup

In [18]:
soup = BeautifulSoup(html_content, 'html.parser')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [19]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [20]:
data = requests.get(team_urls[0])
content2 = data.text

In [21]:
from io import StringIO

In [22]:
import pandas as pd
matches = pd.read_html(StringIO(content2), match="Scores & Fixtures")[0]

In [23]:
soup = BeautifulSoup(content2)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [24]:
data = requests.get(f"https://fbref.com{links[0]}")
content3 = data.text

In [25]:
shooting = pd.read_html(StringIO(content3), match="Shooting")[0]

In [35]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-08-17,15:00,Premier League,Matchweek 1,Sat,Home,D,1,1,Bournemouth,...,19.2,0.0,0,0,1.3,1.3,0.10,-0.3,-0.3,Match Report
1,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Away,W,1,0,Southampton,...,17.2,0.0,0,0,2.2,2.2,0.11,-1.2,-1.2,Match Report
2,2024-08-28,20:00,EFL Cup,Second round,Wed,Home,D,1 (3),1 (4),Newcastle Utd,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
3,2024-08-31,15:00,Premier League,Matchweek 3,Sat,Home,D,1,1,Wolves,...,14.6,1.0,0,0,1.0,1.0,0.06,0.0,0.0,Match Report
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Liverpool,...,18.6,0.0,0,0,0.4,0.4,0.09,0.6,0.6,Match Report


In [26]:
shooting.columns = shooting.columns.droplevel()

In [27]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [36]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2024-08-17,15:00,Premier League,Matchweek 1,Sat,Home,D,1,1,Bournemouth,...,Match Report,NaN,14,8,19.2,0.0,0,0,2024,Nottingham Forest
1,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Away,W,1,0,Southampton,...,Match Report,NaN,23,8,17.2,0.0,0,0,2024,Nottingham Forest
3,2024-08-31,15:00,Premier League,Matchweek 3,Sat,Home,D,1,1,Wolves,...,Match Report,NaN,16,5,14.6,1.0,0,0,2024,Nottingham Forest
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Liverpool,...,Match Report,NaN,5,3,18.6,0.0,0,0,2024,Nottingham Forest
5,2024-09-22,14:00,Premier League,Matchweek 5,Sun,Away,D,2,2,Brighton,...,Match Report,NaN,3,2,17.4,0.0,1,1,2024,Nottingham Forest


In [38]:
years = list(range(2022, 2021, -1))
all_matches = []

In [39]:
years

[2022]

In [40]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [41]:
import time

In [42]:
for year in years:
    content = requests.get(standings_url).text
    soup = BeautifulSoup(content)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        content11 = requests.get(team_url).text
        
        matches = pd.read_html(StringIO(content11), match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(content11)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        content22 = requests.get(f"https://fbref.com{links[0]}").text
        
        shooting = pd.read_html(StringIO(content22), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(5)

IndexError: list index out of range